In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import datetime
import gc
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error


import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb

%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:

def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics

In [ ]:
df_train = import_data('/kaggle/input/testval-processing/df_train.csv')

In [ ]:
df_val = import_data('/kaggle/input/testval-processing/df_val.csv')

In [ ]:
data_test = import_data('/kaggle/input/testval-processing/data_test.csv')
data_test["datetime"] = pd.to_datetime(data_test["timestamp"])
data_test["week"] = data_test["datetime"].dt.week;

# LGBM Model

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_train.primary_use = le.fit_transform(df_train.primary_use)
df_val.primary_use = le.fit_transform(df_val.primary_use)
data_test.primary_use = le.fit_transform(data_test.primary_use)

In [ ]:
df_val.shape[0] #25% of train data for validation

In [ ]:
df_train.shape[0] #75% of train data for the training

In [ ]:
gc.collect()

In [ ]:
y_train = df_train['meter_reading']
y_val = df_val['meter_reading']

df_train = df_train.drop(columns=['meter_reading'])
df_val = df_val.drop(columns=['meter_reading'])

In [ ]:
# Drop timestamp because model does not accept
df_train = df_train.drop(columns=['timestamp'])
df_val = df_val.drop(columns=['timestamp'])
data_test = data_test.drop(columns=['timestamp'])


In [ ]:
from lightgbm import LGBMRegressor, plot_importance
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
# Model
lgbmr = LGBMRegressor(n_estimators=500, random_state=10)
lgbmr.fit(df_train, y_train)
y_predict = lgbmr.predict(df_val)

print(y_predict)

In [ ]:
mse = mse(y_val, y_predict)
print('MSE: %.3f, RMSE: %.3f ' % (mse, sqrt(mse)))


In [ ]:
data_test = data_test.drop(columns=['datetime','row_id'])
data_test.head()

In [ ]:
gc.collect()

In [ ]:
STEP = 1000000
y_test_predict = []
for i in range(0, data_test.shape[0], STEP):
    batch_prediction = lgbmr.predict(data_test.loc[i:i+STEP-1,:])
    y_test_predict.append(list(batch_prediction))
y_test = []
for predictions in y_test_predict:
    y_test = y_test + predictions

In [ ]:
submission = pd.read_csv(f"../input/ashrae-energy-prediction/sample_submission.csv")

# submition data

In [ ]:
data_test['meter_reading']=y_test  #Ajouter les prédictions dans la base de données test
data_to_convtest=data_test[data_test['site_id']==0].index #Convertir les prédictions du site_id  0 
for j in data_to_convtest :
    data_test.at[j,'meter_reading']=(data_test.at[j,'meter_reading']*3.4118)
submission['meter_reading'] = data_test['meter_reading'] 
submission['meter_reading'].clip(lower=0,upper=None,inplace=True) # au cas ou le modèle a prédit une valeur négative
submission.to_csv("No_norm_LGBM_2.csv",index=None) #cette fonction va générer le fichier de soumission avec les prédictions 
submission.head(5)
